##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Classificação de texto com o TensorFlow Hub: Críticas de filmes

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/text_classification_with_hub"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Ver em TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Executar no Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Ver código fonte no GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Baixar notebook</a>
  </td>
</table>


Este notebook classifica as resenhas de filmes como *positivas* ou *negativas* usando o texto da resenha. Este é um exemplo de classificação *binária* - ou de duas classes -, um tipo importante e amplamente aplicável de problema de aprendizado de máquina.

O tutorial demonstra a aplicação básica do aprendizado de transferência com o TensorFlow Hub e o Keras.

Usaremos o [conjunto de dados IMDB] (https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb) que contém o texto de 50.000 críticas de filmes do [Internet Movie Database] (https: //www.imdb.com/). Eles são divididos em 25.000 análises para treinamento e 25.000 análises para testes. Os conjuntos de treinamento e teste são *balanceados*, o que significa que eles contêm um número igual de análises positivas e negativas.

Este notebook usa o [tf.keras] (https://www.tensorflow.org/guide/keras), uma API de alto nível para criar e treinar modelos no TensorFlow e o [TensorFlow Hub] (https: //www.tensorflow .org/hub), uma biblioteca e plataforma para transferência de aprendizado. Para um tutorial de classificação de texto mais avançado usando o `tf.keras`, consulte o [Guia de classificação de texto do MLCC] (https://developers.google.com/machine-learning/guides/text-classification/).

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

!pip install tensorflow-hub
!pip install tensorflow-datasets
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

## Baixar o conjunto de dados IMDB

O conjunto de dados IMDB está disponível em [revisões imdb] (https://www.tensorflow.org/datasets/catalog/imdb_reviews) ou em [conjuntos de dados TensorFlow] (https://www.tensorflow.org/datasets). O código a seguir baixa o conjunto de dados IMDB para sua máquina (ou em tempo de execução colab):

In [0]:
# Divida o conjunto de treinamento em 60% e 40%, para terminar com 15.000 exemplos
# para treinamento, 10.000 exemplos para validação e 25.000 exemplos para teste.
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])

(train_data, validation_data), test_data = tfds.load(
    name="imdb_reviews", 
    split=(train_validation_split, tfds.Split.TEST),
    as_supervised=True)

## Explore os dados

Vamos dedicar um momento para entender o formato dos dados. Cada exemplo é uma frase que representa a crítica de cinema e um rótulo correspondente. A sentença não é pré-processada de forma alguma. O rótulo é um valor inteiro de 0 ou 1, em que 0 é uma revisão negativa e 1 é uma revisão positiva.

Vamos imprimir os 10 primeiros exemplos.

In [0]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

Vamos também imprimir as 10 primeiras etiquetas.

In [0]:
train_labels_batch

## Construindo o modelo

A rede neural é criada empilhando camadas - isso requer três decisões arquiteturais principais:

* Como representar o texto?
* Quantas camadas usar no modelo?
* Quantas *unidades ocultas* para usar em cada camada?

Neste exemplo, os dados de entrada consistem em frases. Os rótulos a serem preditos são 0 ou 1.

Uma maneira de representar o texto é converter frases em vetores de incorporação. Podemos usar uma incorporação de texto pré-treinada como a primeira camada, que terá três vantagens:

* não precisamos nos preocupar com pré-processamento de texto,
* podemos nos beneficiar da transferência de aprendizado,
* a incorporação tem um tamanho fixo, por isso é mais simples de processar.

Neste exemplo, usaremos um **modelo de incorporação de texto pré-treinado** do [TensorFlow Hub] (https://www.tensorflow.org/hub) chamado [google/tf2-preview/gnews-swivel-20dim/1 ] (https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1).

Existem outros três modelos pré-treinados para testar neste tutorial:

* [google/tf2-preview/gnews-swivel-20dim-with-oov/1] (https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1) - o mesmo que [google/tf2-preview/gnews-swivel-20dim/1] (https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1), mas com vocabulário de 2,5% convertido em OOV buckets. Isso pode ajudar se o vocabulário da tarefa e o vocabulário do modelo não se sobrepuserem totalmente.
* [google/tf2-preview/nnlm-en-dim50/1] (https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1) - Um modelo muito maior com ~ 1M de tamanho de vocabulário e 50 dimensões.
* [google/tf2-preview/nnlm-en-dim128/1] (https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1) - Modelo ainda maior com ~ 1M de tamanho e 128 dimensões.

Vamos primeiro criar uma camada Keras que use um modelo do TensorFlow Hub para incorporar as frases e testá-lo em alguns exemplos de entrada. Note que não importa o tamanho do texto de entrada, a forma de saída dos encaixes é: `(num_examples, embedding_dimension)`.

In [0]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

Vamos agora construir o modelo completo:

In [0]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

As camadas são empilhadas sequencialmente para criar o classificador:

1. A primeira camada é uma camada do TensorFlow Hub. Essa camada usa um Modelo salvo pré-treinado para mapear uma frase em seu vetor de incorporação. O modelo de incorporação de texto pré-treinado que estamos usando ([google/tf2-preview/gnews-swivel-20dim/1] (https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1) ) divide a sentença em tokens, incorpora cada token e depois combina a incorporação. As dimensões resultantes são: `(num_examples, embedding_dimension)`.
2. Esse vetor de saída de comprimento fixo é canalizado através de uma camada totalmente conectada (`Densa`) com 16 unidades ocultas.
3. A última camada está densamente conectada com um único nó de saída. Usando a função de ativação `sigmoid`, esse valor é um valor flutuante entre 0 e 1, representando uma probabilidade ou nível de confiança.

Vamos compilar o modelo.

### Função de Perda e Otimizador

Um modelo precisa de uma função de perda e um otimizador para treinamento. Como esse é um problema de classificação binária e o modelo gera uma probabilidade (uma camada de unidade única com uma ativação sigmóide), usaremos a função de perda `binary_crossentropy`.

Esta não é a única opção para uma função de perda; você pode, por exemplo, escolher `mean_squared_error`. Mas, geralmente, `binary_crossentropy` é melhor para lidar com probabilidades - mede a" distância "entre distribuições de probabilidade, ou no nosso caso, entre a distribuição da verdade da terra e as previsões.

Mais tarde, quando estivermos explorando problemas de regressão (por exemplo, para prever o preço de uma casa, veremos como usar outra função de perda denominada erro quadrático médio).

Agora, configure o modelo para usar um otimizador e uma função de perda:

In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Treinar o Modelo

Treine o modelo por 20 épocas em mini-lotes de 512 amostras. São 20 iterações em todas as amostras nos tensores `x_train` e `y_train`. Durante o treinamento, monitore a perda e a precisão do modelo nas 10.000 amostras do conjunto de validação:

In [0]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

## Avaliar o Modelo

E vamos ver como o modelo funciona. Dois valores serão retornados. Perda (um número que representa nosso erro, valores mais baixos são melhores) e precisão.

In [0]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

Essa abordagem bastante ingênua atinge uma precisão de cerca de 87%. Com abordagens mais avançadas, o modelo deve se aproximar de 95%.

## Leitura adicional

Para uma maneira mais geral de trabalhar com entradas de string e para uma análise mais detalhada do progresso da precisão e da perda durante o treinamento, dê uma olhada [aqui] (https://www.tensorflow.org/tutorials/keras/basic_text_classification).